In [ ]:
import requests #Library for fetching data from API
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder #To convert categorical data into numerical values
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta #To handle date and time
import pytz

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()

API_KEY = os.getenv('API_KEY')
BASE_URL = 'https://api.openweathermap.org/data/2.5/'


# **Fetch Current Weather**

In [ ]:
def get_current_weather(city):
  url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric" #API Request URL
  response = requests.get(url)
  data = response.json()
  return{
      'city' : data['name'],
      'current_temperature' : round(data['main']['temp']),
      'feels_like' : round(data['main']['feels_like']),
      'temp_min' : round(data['main']['temp_min']),
      'temp_max' : round(data['main']['temp_max']),
      'humidity' : round(data['main']['humidity']),
      'description' : data['weather'][0]['description'],
      'country' : data['sys']['country'],
      'wind_gust_dir' : data['wind']['deg'],
      'pressure' : data['main']['pressure'],
      'Wind_Gust_Speed' : data['wind']['speed']
  }


# **Read Historical Data**

In [ ]:
def read_historical_data(filename):
  df = pd.read_csv(filename)
  df = df.dropna()
  df = df.drop_duplicates()
  # df['date'] = pd.to_datetime(df['date'])
  return df

# **Prepare data for training**

In [ ]:
def prepare_data(data):
  le = LabelEncoder() #Create a label encoder
  data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
  data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

  X = data[['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed','Humidity', 'Pressure', 'Temp']] #Feature Variable
  y = data['RainTomorrow'] #Target Variable

  return X, y, le

# **Train Prediction Model**

In [ ]:
def train_rain_model(X, y):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  model = RandomForestClassifier(n_estimators=100, random_state = 42)
  model.fit(X_train, y_train) #Training the model

  y_pred = model.predict(X_test)

  accuracy = mean_squared_error(y_test, y_pred)
  print(f"Mean Squared Error for rain model: {accuracy}")

  return model

# **Prepare Regression Data**

In [ ]:
def prepare_regreession_data(data, feature):
  X, y = [], [] #Lists for feature and target values

  for i in range(len(data) - 1):
    X.append(data[feature].iloc[i])

    y.append(data[feature].iloc[i + 1])

  X = np.array(X).reshape(-1, 1)
  y = np.array(y)
  return X, y

# **Train Regression Model**

In [ ]:
def train_regression_model(X, y):
  model = RandomForestRegressor(n_estimators=100, random_state=42)
  model.fit(X, y) #Train the model

  return model

# **Predict Future**

In [ ]:
def predict_future(model, current_value):
  predictions = [current_value]

  for i in range(5):
    next_value = model.predict(np.array([[predictions[-1]]]))

    predictions.append(next_value[0])

  return predictions[1:]


# **Weather Analysis Function**

In [ ]:
def weather_view():
  city = input("Enter any city name: ")
  current_weather = get_current_weather(city)

  #load historical data
  historical_data = read_historical_data('/content/weather.csv')

  #prepare and train the rain prediction model
  X, y, le = prepare_data(historical_data)

  rain_model = train_rain_model(X, y)

  #Map wind direction to compass points(because the API provides numerical values only)

  wind_deg = current_weather['wind_gust_dir'] % 360
  compass_points = [
      ("N", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25), ("ENE", 56.25, 78.75),
      ("E", 78.75, 101.25), ("ESE", 101.25, 123.75), ("SE", 123.75, 146.25), ("SSE",146.25, 168.75),
      ("S", 168.75, 191.25), ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
      ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25), ("NNW", 326.25, 348.75)
  ]
  compass_direction = next(direction for direction, start, end in compass_points if start <= wind_deg < end)

  compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1


  current_data = {
      'MinTemp' : current_weather['temp_min'],
      'MaxTemp' : current_weather['temp_max'],
      'WindGustDir' : compass_direction_encoded,
      'WindGustSpeed' : current_weather['Wind_Gust_Speed'],
      'Humidity' : current_weather['humidity'],
      'Pressure' : current_weather['pressure'],
      'Temp' : current_weather['current_temperature']
  }

  current_df = pd.DataFrame([current_data])

  #Rain Prediction

  rain_prediction = rain_model.predict(current_df)[0]

  #Prepare regression model for temperature and humididty

  X_temp, y_temp = prepare_regreession_data(historical_data, 'Temp')
  X_hum, y_hum = prepare_regreession_data(historical_data, 'Humidity')

  temp_model = train_regression_model(X_temp, y_temp)
  hum_model = train_regression_model(X_hum, y_hum)

  #Predict future temperature and Himidity

  future_temp = predict_future(temp_model, current_weather['temp_min'])

  future_hum = predict_future(hum_model, current_weather['humidity'])

  #Prepare time for future predictions
  timezone = pytz.timezone('Asia/Kolkata')
  now = datetime.now(timezone)
  next_hour = now + timedelta(hours=1)
  next_hour = next_hour.replace(minute=0, second=0, microsecond=0)

  future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

  #Display results

  print(f"City: {city}, {current_weather['country']}")
  print(f"Current Temperature: {current_weather['current_temperature']}°C")
  print(f"Feels Like: {current_weather['feels_like']}°C")
  print(f"Minimum Temperature: {current_weather['temp_min']}°C")
  print(f"Maximum Temperature: {current_weather['temp_max']}°C")
  print(f"Humidity: {current_weather['humidity']}%")
  print(f"Weather Prediction: {current_weather['description']}")
  print(f"Rain Prediction: {'Yes' if rain_prediction else 'No'}")

  print("\n Future Temperature Predictions:")

  for time, temp in zip(future_times, future_temp):
    print(f"At {time}: {round(temp, 1)}°C")

  print("\n Future Humidity Predictions:")

  for time, humidity in zip(future_times, future_hum):
    print(f"At {time}: {round(humidity, 1)}%")


weather_view()


Enter any city name: Bangalore
Mean Squared Error for rain model: 0.1506849315068493
City: Bangalore, IN
Current Temperature: 28°C
Feels Like: 30°C
Minimum Temperature: 28°C
Maximum Temperature: 29°C
Humidity: 61%
Weather Prediction: overcast clouds
Rain Prediction: Yes

 Future Temperature Predictions:
At 18:00: 27.7°C
At 19:00: 22.8°C
At 20:00: 25.4°C
At 21:00: 24.7°C
At 22:00: 23.4°C

 Future Humidity Predictions:
At 18:00: 58.3%
At 19:00: 54.2%
At 20:00: 58.1%
At 21:00: 54.2%
At 22:00: 58.1%
